# Use Splash to Request ZocDoc Information

In [1]:
from scrapy.http import TextResponse
import requests
from urllib.parse import urlencode


# you need start the docker
# url = 'https://www.zocdoc.com/doctor/ghulam-abbas-md-291989'
url = 'https://www.zocdoc.com/doctor/dane-salazar-md-246564'

# r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait':0})
proxyapi = '68cd99bcfa3c0879b87ea3052f61da4e'
r = requests.get('http://api.scraperapi.com/', params=urlencode({'api_key': proxyapi, 'url': url}))

response = TextResponse(r.url, body = r.text, encoding = 'utf-8')


In [2]:
response.text

'<!doctype html>\n    <html lang="en">\n        <head>\n            <meta name="msapplication-TileImage" content="/images/rebrand/favicon/favicon_large.png" />\n<meta charset="utf-8" />\n<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" />\n<meta name="format-detection" content="telephone=no" />\n            <link rel="shortcut icon" type="image/x-icon" href="https://www.zocdoc.com/favicon.ico" sizes="32x32" />\n<link rel="dns-prefetch" href="//api.zocdoc.com" />\n<link rel="dns-prefetch" href="//www.zocdoc.com" />\n<link rel="preconnect" href="https://api.zocdoc.com" />\n<link rel="preconnect" href="https://www.zocdoc.com" />\n<link rel="dns-prefetch" href="//d1uhlocgth3qyq.cloudfront.net" />\n<link rel="preconnect" href="https://d1uhlocgth3qyq.cloudfront.net" crossorigin="anonymous" />\n<link rel="dns-prefetch" href="//connect.facebook.net" />\n<link rel="preconnect" href="https://connect.facebook.net" crossorigin="anonymous" />\n<

# Json Data

In [3]:
import json
from pprint import pprint
xpath = './/script//text()'

json_data = response.xpath(xpath).extract()
len(json_data)

5

In [4]:
json_string = [i for i in json_data if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]
json_info = [i for i in json_string.split(';\n') if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]

json_info = json_info.split('JSON.parse(')[1][:-1] # remove prefix
# x = x[:-1] # remove suffix
# json_info



x = json_info
x = json.loads(x)
x = json.loads(x)
pprint(x)

{'@@locationsCarousel': {},
 '@@mobileProfileBookingModal': {'shouldShowMobileProfileBookingModal': False},
 '@@pictureGallery': {'currentPicture': 0,
                      'pictures': [],
                      'showPictures': False},
 '@@profile/availability': {'nextAvailability': {'lo_7RbDan3KSE6HP4x-Zsk2eR': '2021-11-08T07:50:00-06:00',
                                                 'lo_GbP9QOoeVkSKYVy5NI5auB': '2021-11-04T13:50:00-05:00',
                                                 'lo_I4iTXVC7H02FHKdQRNmv7B': '2021-11-04T15:50:00-05:00',
                                                 'lo_POasOqE2jEOIDE5UAWqKrx': '2021-11-08T09:20:00-06:00',
                                                 'lo_Zevx4YN5PEGyuropNvfpax': '2021-11-03T07:50:00-05:00',
                                                 'lo_cabGRzy11Ua0qDopWIWnkB': None,
                                                 'lo_kAmUzioUuEWnz-Zzr_eO5x': '2021-11-01T13:40:00-05:00',
                                       

                                                                      'locations': [],
                                                                      'name': 'Lumenos '
                                                                              'Plans '
                                                                              '(PPO)'},
                                                                     {'__typename': 'InsurancePlan',
                                                                      'id': 'ip_3918',
                                                                      'locations': [],
                                                                      'name': 'PPO '
                                                                              'Share '},
                                                                     {'__typename': 'InsurancePlan',
                                                                      'id': 'ip_4497',
            

                                                                      'locations': [],
                                                                      'name': 'Blue '
                                                                              'Cross '
                                                                              'Blue '
                                                                              'Shield '
                                                                              'PPO '
                                                                              'Plus'},
                                                                     {'__typename': 'InsurancePlan',
                                                                      'id': 'ip_16497',
                                                                      'locations': [],
                                                                      'name': 'Blue '
                                

                                                                      'id': 'ip_7803',
                                                                      'locations': [],
                                                                      'name': 'Eye '
                                                                              'Exam '
                                                                              'Only '
                                                                              'Plan '
                                                                              '( '
                                                                              'Large '
                                                                              'Group '
                                                                              'only)'},
                                                                     {'__typename': 'InsurancePlan',
                                     

                                                                      'locations': [],
                                                                      'name': 'Veterans '
                                                                              'Choice '
                                                                              'Program'}]},
                                                          {'__typename': 'InsuranceCarrier',
                                                           'id': 'ic_461',
                                                           'name': 'Healthlink',
                                                           'plans': [{'__typename': 'InsurancePlan',
                                                                      'id': 'ip_8552',
                                                                      'locations': [],
                                                                      'name': 'HealthLink '
                           

                                                                      'locations': [],
                                                                      'name': 'Care '
                                                                              'Improvement '
                                                                              'Plus '
                                                                              'Medicare '
                                                                              'Advantage '
                                                                              '(Regional '
                                                                              'PPO)'},
                                                                     {'__typename': 'InsurancePlan',
                                                                      'id': 'ip_4657',
                                                                      'locations': [],
              

                                                                                      'date': '2021-11-04',
                                                                                      'timeslots': []},
                                                                                     {'__typename': 'TimeslotDay',
                                                                                      'date': '2021-11-05',
                                                                                      'timeslots': []},
                                                                                     {'__typename': 'TimeslotDay',
                                                                                      'date': '2021-11-06',
                                                                                      'timeslots': []},
                                                                                     {'__typename': 'TimeslotDay',
                   

                                                               'patients on '
                                                               'Zocdoc. <a '
                                                               'href="https://www.zocdoc.com/doctor/dane-salazar-md-246564?utm_source=google&utm_medium=search&utm_campaign=faq-profile&utm_content=profile-2" '
                                                               'rel="nofollow"> '
                                                               'You can see '
                                                               "Dr. Salazar's "
                                                               'earliest '
                                                               'availability '
                                                               '</a> on Zocdoc '
                                                               'and schedule '
                                                               'an appointment '
 

                               'question': '<h3>How soon can I make an '
                                           'appointment with Dr. Dane '
                                           'Salazar?</h3>'},
                              {'__typename': 'FAQ',
                               'answer': '<p> Dr. Dane Salazar generally '
                                         'accepts new patients on Zocdoc. <a '
                                         'href="https://www.zocdoc.com/doctor/dane-salazar-md-246564?utm_source=google&utm_medium=search&utm_campaign=faq-profile&utm_content=profile-2" '
                                         'rel="nofollow"> You can see Dr. '
                                         "Salazar's earliest availability </a> "
                                         'on Zocdoc and schedule an '
                                         'appointment online.</p>',
                               'question': '<h3>Is Dr. Dane Salazar accepting '
                          

                                                                                                'availability '
                                                                                                'for '
                                                                                                'new '
                                                                                                'patients',
                                                                    'NoNewPatientAvailabilityWithCta': 'New '
                                                                                                       'patient? '
                                                                                                       'Call '
                                                                                                       '{phoneNumber} '
                                                                                                       'for '
        

In [5]:
doc_info = {}

## Root Info

In [6]:
[# 'routing',
 # 'theme',
 # 'absystem',
 # '@location',
 # 'pageload',
 # '@@viewport',
 # 'prefetch',
 # 'user',
#  'signInForm',
 # '@@profile/in_network_insurance',
 # 'insurance-picker',
 # 'insurance',
 # '@@locationsCarousel',
 # 'NewExistingRadio',
 # '@@pictureGallery',
 'profile',
#  'seo',
 # 'patientPoweredSearch',
 # 'search',
 # '@@quickLinks',
 # 'timesgrid',
 # '@@videoVisitExplainerModal',
 # '@@profile/availability',
 # '@@mobileProfileBookingModal',
 # 'cookies'
]


# doc_info['timesgrid'] = x['timesgrid']
doc_info

{}

## Profile Info

In [7]:
col = 'profile'

profile = x[col]
[i for i in profile]
# x[col]

['bookingSpotlightIsOn',
 'carrierId',
 'expandedLocations',
 'initParameters',
 'mapIsActive',
 'planId',
 'data',
 'dataVariables',
 'providerId',
 'deviceType',
 'provider',
 'isNewPatient',
 'isMentalHealthProvider',
 'doesProviderHaveInNetworkInsurances',
 'selectedLocationId',
 'procedureId',
 'commonVirtualVisitReasons',
 'videoVisitExplainerModalType',
 'customDimensions',
 'defaultProcedureId',
 'eventData',
 'host',
 'pageId',
 'pageCategory',
 'prefix',
 'seo',
 'trustedInsuranceRating',
 'brandingVanityUrl']

In [8]:
cols = [# 'bookingSpotlightIsOn',
 # 'carrierId',
 # 'expandedLocations',
 # 'initParameters',
 # 'mapIsActive',
 # 'planId',
 # 'data',    # 'data',
 # 'dataVariables',
 'providerId',
 # 'deviceType',
#  'provider', # 'provider'
 # 'isNewPatient ',
 'isMentalHealthProvider',
 'doesProviderHaveInNetworkInsurances',
 # 'selectedLocationId',
#  'procedureId',
 'commonVirtualVisitReasons',
 # 'videoVisitExplainerModalType',
 'customDimensions',
 # 'defaultProcedureId',
#  'eventData',
 # 'host',
 # 'pageId',
 # 'pageCategory',
 # 'prefix',
 # 'seo',
#  'trustedInsuranceRating',
 'brandingVanityUrl']


for col in cols: doc_info[col] = profile[col]
    
pprint(doc_info)


{'brandingVanityUrl': None,
 'commonVirtualVisitReasons': {'commonVisitReasonIds': [75,
                                                        83,
                                                        77,
                                                        1309,
                                                        494,
                                                        84,
                                                        86,
                                                        1005,
                                                        129,
                                                        466,
                                                        1063,
                                                        3194,
                                                        2366,
                                                        111,
                                                        3193,
                                                        

### Profile Info - DataProvider

In [9]:
profile_data_provider = profile['data']['provider']
# [i for i in profile_data_provider]

In [10]:
cols = ['isOnlyInNetworkBookable','acceptsNewPatients','errorCode','id','monolithId','status',
 'profileUrl','isApproved','prenominal','postnominal','firstName',
 'lastName','directoryIds', 'approvedFullName', 'nameInSentence','newPatientAppointmentsAsIntroCalls',
 
 'genderIdentity', 'frontEndCirclePictureUrl', 'frontEndSquarePictureUrl', 'showPhotoWithCircleCss', 'smallPictures',
 'averageRating', 'averageBedsideRating','averageWaitTimeRating',
 'isPreview','personIdentifierSource', 'dataProvidedBy', 'dataProvidedByUrl',
 
 'mayChargeAdditionalCopay','onlySeesChildren', 'optOutOfRatings', 'previewSummary',
 'highlyRecommendPercentage', 'lowWaitTimedPercentage', 'hasNewPatientAvailability',
 'isTopRebookingProvider', 'hasVirtualLocations',
 'selfPaySettings', 'sellingPoints', 'approvedNonVirtualLocations', 'practices',
 'providerLocations', 'relevantSpecialty', 'mainSpecialtyId','specialties','approvedLocations',
 

 'statement', 'defaultProcedureId', 'hospitalAffiliations', 'certifications', 'education',
 'memberships', 'typedMemberships','awards','badges', 'languages','npi',
 'inNetworkInsurance','procedures','popularProcedures','isBookable','seo','reviewCount',
 'reviews', 'properties', 'trustedInsuranceRating', 'providerBadges','locations',
 'sortedLocations', 'numVirtualLocations',
 #  '__typename',
 # 'recentReviews',
]


# l = profile_data_provider['isOnlyInNetworkBookable']
# pprint(l)



for col in cols: doc_info[col] = profile_data_provider[col]
    
    
pprint(doc_info)
    

{'acceptsNewPatients': True,
 'approvedFullName': 'Dr. Dane Salazar, MD',
 'approvedLocations': [{'__typename': 'Location',
                        'address1': '2160 S 1st Ave',
                        'address2': '',
                        'city': 'Maywood',
                        'fax': '',
                        'id': 'lo_7RbDan3KSE6HP4x-Zsk2eR',
                        'isVirtual': False,
                        'latitude': 41.8605338,
                        'longitude': -87.8350244,
                        'monolithId': '102244',
                        'name': 'Loyola Outpatient Center',
                        'officeHours': [],
                        'officeHoursDict': None,
                        'phone': '(888) 584-7888',
                        'phoneExt': '',
                        'practice': {'__typename': 'Practice',
                                     'id': 'pt_AMyGKbT3rkWcSWt7g_pIRh',
                                     'name': 'Loyola Medicine - Orthopedics'}

                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_9504',
                                    'locations': [],
                                    'name': 'Blue Access Network (Indiana)'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_9508',
                                    'locations': [],
                                    'name': 'Blue Access Network (Kentucky)'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_9511',
                                    'locations': [],
                                    'name': 'Blue Access Network (Missouri)'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_9535',
                                    'locations': [],
                                    'name': 'Blue Access Network (Ohio)'

                                            '(Wisconsin)'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_10109',
                                    'locations': [],
                                    'name': 'ME: Health Choice (Health Choice '
                                            'PPO)'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_10114',
                                    'locations': [],
                                    'name': 'MaineHealth Healthy Saver'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_12651',
                                    'locations': [],
                                    'name': 'MemorialCare Custom PPO'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_10124',
                  

                                    'name': 'BlueCard PPO Basic'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_5580',
                                    'locations': [],
                                    'name': 'BlueCard Traditional'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_5577',
                                    'locations': [],
                                    'name': 'Mammography (Routine Network)'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_5567',
                                    'locations': [],
                                    'name': 'Medicare Advantage'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_5568',
                                    'locations': [],
                           

                         'name': 'Highmark Blue Cross Blue Shield',
                         'plans': [{'__typename': 'InsurancePlan',
                                    'id': 'ip_11760',
                                    'locations': [],
                                    'name': 'Advance Blue'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_15993',
                                    'locations': [],
                                    'name': 'AffordaBlue'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_4056',
                                    'locations': [],
                                    'name': 'BCBS PPO'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_13297',
                                    'locations': [],
                                    'name': 'Balance Blue P

                                    'name': 'SuperBlue Select'}]},
                        {'__typename': 'InsuranceCarrier',
                         'id': 'ic_314',
                         'name': 'Horizon Blue Cross Blue Shield of New Jersey',
                         'plans': [{'__typename': 'InsurancePlan',
                                    'id': 'ip_8285',
                                    'locations': [],
                                    'name': 'BlueCard PPO'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_13823',
                                    'locations': [],
                                    'name': 'Direct Access Gold 100/80/60 '
                                            'BlueCard'},
                                   {'__typename': 'InsurancePlan',
                                    'id': 'ip_3735',
                                    'locations': [],
                                    'n

                'address2': '',
                'city': 'Burr Ridge',
                'fax': '',
                'id': 'lo_I4iTXVC7H02FHKdQRNmv7B',
                'isVirtual': True,
                'latitude': 41.7595836,
                'longitude': -87.91425009999999,
                'monolithId': '130296',
                'name': '',
                'officeHours': [],
                'officeHoursDict': None,
                'phone': '(888) 584-7888',
                'phoneExt': '',
                'practice': {'__typename': 'Practice',
                             'id': 'pt_AMyGKbT3rkWcSWt7g_pIRh',
                             'name': 'Loyola Medicine - Orthopedics'},
                'properties': {},
                'state': 'IL',
                'virtualVisitType': 'ThirdPartyVideoVisit',
                'zipCode': '60527'},
               {'__typename': 'Location',
                'address1': '2160 S 1st Ave',
                'address2': '',
                'city': 'Maywood',
  

                 'insuranceMayNotCover': False,
                 'name': 'Frozen Shoulder'},
                {'__typename': 'Procedure',
                 'id': '3823',
                 'insuranceMayNotCover': False,
                 'name': "Golfer's Elbow / Medial Epicondylitis"},
                {'__typename': 'Procedure',
                 'id': '3972',
                 'insuranceMayNotCover': False,
                 'name': 'Independent Medical Exam (Orthopedic)'},
                {'__typename': 'Procedure',
                 'id': '3233',
                 'insuranceMayNotCover': False,
                 'name': 'Joint Dislocations'},
                {'__typename': 'Procedure',
                 'id': '1800',
                 'insuranceMayNotCover': False,
                 'name': 'Joint Injection'},
                {'__typename': 'Procedure',
                 'id': '3820',
                 'insuranceMayNotCover': False,
                 'name': 'Labral Tear'},
                {'__type

              'isMailedInReview': True,
              'isPartnerReview': False,
              'isVirtualVisit': False,
              'overallRating': 5,
              'patientName': 'DO',
              'reviewId': 're_tz_91QMiUkCIFCg8OnyGFh',
              'waitTimeRating': 5},
             {'__typename': 'Review',
              'bedsideRating': 5,
              'comment': 'I had been having issues with my shoulder for over '
                         '15 years before meeting Dr. Salazar. I had consulted '
                         'over 5 surgeons to end the constant suffering of '
                         'pain and limitation of mobility. Dr. Salazar’s '
                         'professional approach, patience and follow up after '
                         'each stage, before and after the total shoulder '
                         'replacement surgery, ended my suffering and now I '
                         'live normally and am able to use my arm without any '
                       

              'dateRange': 'more_than_1_year',
              'helpfulCount': 0,
              'isMailedInReview': False,
              'isPartnerReview': False,
              'isVirtualVisit': True,
              'overallRating': 1,
              'patientName': '',
              'reviewId': 're_W0S6X9YBjUWZMkMvMu1nyx',
              'waitTimeRating': 0},
             {'__typename': 'Review',
              'bedsideRating': 5,
              'comment': '',
              'date': '2020-05-22',
              'dateRange': 'unknown',
              'helpfulCount': 0,
              'isMailedInReview': False,
              'isPartnerReview': False,
              'isVirtualVisit': False,
              'overallRating': 5,
              'patientName': '',
              'reviewId': 're_kqr5eStkPE-lTiWrjpIFTR',
              'waitTimeRating': 5},
             {'__typename': 'Review',
              'bedsideRating': 0,
              'comment': '',
              'date': '',
              'dateRange': 'mo

In [11]:
print([i for i in doc_info])

['providerId', 'isMentalHealthProvider', 'doesProviderHaveInNetworkInsurances', 'commonVirtualVisitReasons', 'customDimensions', 'brandingVanityUrl', 'isOnlyInNetworkBookable', 'acceptsNewPatients', 'errorCode', 'id', 'monolithId', 'status', 'profileUrl', 'isApproved', 'prenominal', 'postnominal', 'firstName', 'lastName', 'directoryIds', 'approvedFullName', 'nameInSentence', 'newPatientAppointmentsAsIntroCalls', 'genderIdentity', 'frontEndCirclePictureUrl', 'frontEndSquarePictureUrl', 'showPhotoWithCircleCss', 'smallPictures', 'averageRating', 'averageBedsideRating', 'averageWaitTimeRating', 'isPreview', 'personIdentifierSource', 'dataProvidedBy', 'dataProvidedByUrl', 'mayChargeAdditionalCopay', 'onlySeesChildren', 'optOutOfRatings', 'previewSummary', 'highlyRecommendPercentage', 'lowWaitTimedPercentage', 'hasNewPatientAvailability', 'isTopRebookingProvider', 'hasVirtualLocations', 'selfPaySettings', 'sellingPoints', 'approvedNonVirtualLocations', 'practices', 'providerLocations', 'rel

In [ ]:
len(doc_info['reviews'])

In [ ]:
doc_info['complete_reviews'] = False if doc_info['reviewCount'] > len(doc_info['reviews']) else True

In [ ]:
doc_info['reviews']

# POST: URL (Failed)

In [ ]:

# import requests


# review_api_url = 'https://api.zocdoc.com/directory/v2/gql'
# providerID = 'pr_mw0z_x5HqE-YgjiprYcWax' # TODO
# limit = 100 # TODO

       
# forms = {
#     'operationName': 'getProviderReviews',
#     'query': "query getProviderReviews($providerId: String, $directoryId: String, $limit: Int, $sortReviewsBy: ReviewsSortBy, $sortReviewsDescending: Boolean) {\n  provider(id: $providerId, directoryId: $directoryId) {\n    id\n    ...providerReviews\n    __typename\n  }\n}\n\nfragment providerReviews on Provider {\n  reviews(limit: $limit, sortBy: $sortReviewsBy, sortDescending: $sortReviewsDescending) {\n    reviewId\n    bedsideRating\n    comment\n    date\n    dateRange\n    helpfulCount\n    isMailedInReview\n    isPartnerReview\n    isVirtualVisit\n    overallRating\n    patientName\n    waitTimeRating\n    __typename\n  }\n  __typename\n}\n", 
#     'variables':{
#         'directoryId': "-1",
#         "limit": str(limit),
#         "providerId": providerID,
#         "sortReviewsBy":"date",
#         "sortReviewsDescending": 'true',
#     }
# }


# r = requests.post('http://localhost:8050/render.html',
#               json={'url': review_api_url,
#                     'http_method': 'POST',
#                     'body': forms,
#                     'wait': 2})

# print(r.text)

# Selenium (when reviews > 50)

In [ ]:
# select your webdriver
from scrapy.http import HtmlResponse
from selenium import webdriver
import time

REVIEW_PER_PAGE = 50
webdriver_path = '_source/chromedriver_macm1'
reviewCount = doc_info['reviewCount']
    
    
    
def get_review(review_selector):
    d = {}
    d['comment'] = review_selector.xpath('.//*[@itemprop="reviewBody"]//span/text()').extract_first()
    d['date'] = review_selector.xpath('.//*[@itemprop="datePublished"]/text()').extract_first()
    d['patientName'] = review_selector.xpath('.//*[@itemprop="author"]/text()').extract_first()
    d['patient_type'] = review_selector.xpath('.//*[@data-test="review-source"]/text()').extract_first()
    d['overallRating'] = review_selector.xpath('.//*[@data-test="stars-svg-wrapper"]/svg/@data-rating').extract_first()
    return d


def get_complete_reviews_from_selenium(url, reviewCount, REVIEW_PER_PAGE, webdriver_path):
    options = webdriver.ChromeOptions()
    # options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument('--headless')

    driver = webdriver.Chrome(webdriver_path, options=options)
    driver.get(url)
    driver.title
    time.sleep(2)
    
    xpath = './/button[@data-test="reviews-read-more-button"]'
    
    for _ in range(int(reviewCount / REVIEW_PER_PAGE )):
        more_result_button = driver.find_element_by_xpath(xpath)
        print(more_result_button)
        driver.execute_script("arguments[0].click();", more_result_button)
        time.sleep(1)
        
    response = HtmlResponse(driver.current_url, body=driver.page_source, encoding='utf-8')
    reviews = response.xpath('//div[@itemprop="reviews"]')
    reviews = [get_review(i) for i in reviews]
    # len(reviews)
    driver.quit()
    return reviews



reviews = get_complete_reviews_from_selenium(url, reviewCount, REVIEW_PER_PAGE, webdriver_path)

print('\nReviews: reported {} vs collected {}'.format(reviewCount, len(reviews)))



# All In One

In [ ]:
import os
import requests
import pandas as pd
import time
import requests
from scrapy.http import TextResponse
import json
import html
import argparse
import random
from datetime import datetime
from scrapy.http import HtmlResponse
from selenium import webdriver
import time
from datetime import datetime



headers = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36",
}
        
def get_review(review_selector):
    d = {}
    d['comment'] = review_selector.xpath('.//*[@itemprop="reviewBody"]//span/text()').extract_first()
    d['date'] = review_selector.xpath('.//*[@itemprop="datePublished"]/text()').extract_first()
    d['patientName'] = review_selector.xpath('.//*[@itemprop="author"]/text()').extract_first()
    d['patient_type'] = review_selector.xpath('.//*[@data-test="review-source"]/text()').extract_first()
    d['overallRating'] = review_selector.xpath('.//*[@data-test="stars-svg-wrapper"]/svg/@data-rating').extract_first()
    return d

def get_complete_reviews_from_selenium(url, reviewCount, REVIEW_PER_PAGE, webdriver_path):
    options = webdriver.ChromeOptions()
    # options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument('--headless')

    driver = webdriver.Chrome(webdriver_path, options=options)
    driver.get(url)
    driver.title
    time.sleep(2)
    
    xpath = './/button[@data-test="reviews-read-more-button"]'
    
    for _ in range(int(reviewCount / REVIEW_PER_PAGE )):
        more_result_button = driver.find_element_by_xpath(xpath)
        print(more_result_button)
        driver.execute_script("arguments[0].click();", more_result_button)
        time.sleep(1)
        
    response = HtmlResponse(driver.current_url, body=driver.page_source, encoding='utf-8')
    reviews = response.xpath('//div[@itemprop="reviews"]')
    reviews = [get_review(i) for i in reviews]
    # len(reviews)
    driver.quit()
    return reviews


def get_physician_info_from_zocdoc_url(ph_url, REVIEW_PER_PAGE, webdriver_path, use_webdriver):
    doc_info = {}
    
    r = requests.get('http://localhost:8050/render.html', params={'url': ph_url, 'wait':0})
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')
    
    xpath = './/script//text()'

    json_data = response.xpath(xpath).extract()
    # len(json_data)

    json_string = [i for i in json_data if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]
    json_info = [i for i in json_string.split(';\n') if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]

    json_info = json_info.split('JSON.parse(')[1][:-1] # remove prefix
    # x = x[:-1] # remove suffix
    # json_info

    x = json_info
    x = json.loads(x)
    x = json.loads(x) # yes, double json.load operations


    # (1) basic profile
    col = 'profile'; profile = x[col]

    cols = [ 'providerId', 'isMentalHealthProvider',
        'doesProviderHaveInNetworkInsurances', 
        'commonVirtualVisitReasons', 'customDimensions', 'brandingVanityUrl']

    for col in cols: doc_info[col] = profile[col]

    # (2) provider
    profile_data_provider = profile['data']['provider']

    cols = ['isOnlyInNetworkBookable','acceptsNewPatients','errorCode','id','monolithId','status',
        'profileUrl','isApproved','prenominal','postnominal','firstName',
        'lastName','directoryIds', 'approvedFullName', 'nameInSentence','newPatientAppointmentsAsIntroCalls',
        'genderIdentity', 'frontEndCirclePictureUrl', 'frontEndSquarePictureUrl', 'showPhotoWithCircleCss', 'smallPictures',
        'averageRating', 'averageBedsideRating','averageWaitTimeRating',
        'isPreview','personIdentifierSource', 'dataProvidedBy', 'dataProvidedByUrl',
        'mayChargeAdditionalCopay','onlySeesChildren', 'optOutOfRatings', 'previewSummary',
        'highlyRecommendPercentage', 'lowWaitTimedPercentage', 'hasNewPatientAvailability',
        'isTopRebookingProvider', 'hasVirtualLocations',
        'selfPaySettings', 'sellingPoints', 'approvedNonVirtualLocations', 'practices',
        'providerLocations', 'relevantSpecialty', 'mainSpecialtyId','specialties','approvedLocations',
        'statement', 'defaultProcedureId', 'hospitalAffiliations', 'certifications', 'education',
        'memberships', 'typedMemberships','awards','badges', 'languages','npi',
        'inNetworkInsurance','procedures','popularProcedures','isBookable','seo','reviewCount',
        'reviews', 'properties', 'trustedInsuranceRating', 'providerBadges','locations',
        'sortedLocations', 'numVirtualLocations',]

    for col in cols: doc_info[col] = profile_data_provider[col]

    doc_info['complete_reviews'] = False if doc_info['reviewCount'] > len(doc_info['reviews']) else True


    reviewCount = doc_info['reviewCount']

    if doc_info['complete_reviews'] == False and use_webdriver == True:
        reviews = get_complete_reviews_from_selenium(ph_url, reviewCount, 
                                                    REVIEW_PER_PAGE, webdriver_path)

        print('\nReviews: reported {} vs collected {}'.format(reviewCount, len(reviews)))
        if len(reviews) == reviewCount: doc_info['complete_reviews'] = True 
        
        doc_info['reviews'] = reviews
    
    return doc_info


In [ ]:
ph_url = url
REVIEW_PER_PAGE = 50
webdriver_path = '_source/chromedriver_macm1'

doc_info = get_physician_info_from_zocdoc_url(ph_url, REVIEW_PER_PAGE, webdriver_path)

In [ ]:
len(doc_info['reviews'])

In [ ]:
doc_info['reviews']